In [ ]:
# Apart from searching for the smallest vector in a lattice, one can also try to find a closest lattice vector to a random point
# Actually these problems are in some sense equally hard (they are reducible to each other)


# If we have a CVP oracle, we create lattices of the form (b_1,...,b_i-1, 2*b_i, b_i+1,...,b_n)
# find a closest vector to b_i (=x_i) and we choose the shortest among x_i+b_i
# such vector is clearly in the lattice
# the shortest vector is a linear combination with at least one odd coeficient - say a_i - thus it cannot be in the lattice
# (b_1,...,b_i-1, 2*b_i, b_i+1,...,b_n) - the CVP algorithm will return a nonzero vector.

# the SVP to CVP is also doable, but not as easy (why closest vector to 0 does not work?)

In [ ]:
# There are numerous algorithms for getting "short" lattice vectors 
# So far it seems, you can get a not so short vector quickly (LLL), or you can get a short vector slowly
# This time it will (mostly) be about the second approach

In [17]:
# Enumeration based algorithms

# The idea is simple: try all vectors below a given bound. Return the shortest one
# Running time usually strongly depends on the given bound. It is a good idea to at least LLL reduce the basis to get a better bound.


from sage.modules.free_module_integer import IntegerLattice

A = random_matrix(ZZ,3,5)
pretty_print(A)
L = IntegerLattice(A)
B = A.LLL()
pretty_print(A.LLL())
print (A[0].norm())
print (B[0].norm())


print (L.shortest_vector())



# a very simple implementation can look like this

R = rank(A)
bound = floor(B[0].norm())


# actually looking at a hypercube and not a hypersphere
# could thus be optimized
# moreover we could be using group symmetries to further speed up the search

# (if v in L, then -v in L)
test_vectors = cartesian_product([range(-bound,bound)]*5)

for vec in test_vectors:
    # if vec.norm() > bound:
    #    continue
    if rank(B.stack(vector(vec))) == R:
        print (vec)

        
# and we should also keep our best current bound
# and only to the gaussian elimination for shorter vectors



[ 5  0 -1 -1  0]
[ 2  3  0 -1 -1]
[-1  7  1 -4  0]

[ 0  1  0 -3  2]
[ 2  3  0 -1 -1]
[ 3 -3 -1  0  1]

3*sqrt(3)
sqrt(14)
(0, 1, 0, -3, 2)
(-2, -3, 0, 1, 1)
(0, 0, 0, 0, 0)
(0, 1, 0, -3, 2)
(2, 2, 0, 2, -3)


In [ ]:
# The above is not too bad for (nearly) full rank lattices
# However, its efficiency is abysmal - imagine a 2-dimensional lattice in a 120-dimensional space
# solution can be found efficiently by LLL, however, the above_style enumeration, would run very slowly

# fortunately it is not too hard to adapt enumeration even to this case
# we will not be bruteforcing coordinates, but the coefficients of linear combinations of basis vectors
# these can actually be bounded, so the algorithm is well defined
# (e.g. https://www.iacr.org/archive/eurocrypt2010/66320257/66320257.pdf)

In [18]:
# Sieving algorithms

# consist of 2 steps
# 1. collect random vectors
# 2. recombine them into vectors of smaller norm
# (simple, as a lattice is a group)

# a simple implementation could look like this

# sage has uniq function, but it does not work on "mutable vectors", so we need to write our own
def remove_duplicates(l):
    new = []
    for v in l:
        if v not in new:
            new.append(v)
    return new


A = random_matrix(ZZ,3,5)
pretty_print(A)
L = IntegerLattice(A)
B = A.LLL()
pretty_print(A.LLL())
print (A[0].norm())
print (B[0].norm())

# sage implements a sampling procedure
from sage.stats.distributions.discrete_gaussian_lattice import DiscreteGaussianDistributionLatticeSampler

D = DiscreteGaussianDistributionLatticeSampler(A, 10.0)

samples = [D() for _ in range(2^5)]

print(samples)

bound = max([v.norm() for v in samples])/2

while len(samples) > 1:
    samples = [v1-v2 for v1 in samples for v2 in samples if (v1-v2).norm() < bound and v1 != v2]
    samples = remove_duplicates(samples)
    #samples = [sample for sample in samples if sample[0]>=0]
    print(samples)
    bound /= 2
    
    
print (L.shortest_vector())   





[  0  15   1   1 -10]
[  1   1  -3  -1   1]
[ -2   0   0  -1   3]

[ 1  1 -3 -1  1]
[-2  0  0 -1  3]
[-4 15  1 -1 -4]

sqrt(327)
sqrt(13)
[(5, 14, 4, 5, -20), (6, 4, -12, -3, 1), (4, -2, 6, 5, -11), (-9, -14, -4, -7, 26), (2, 2, -6, -2, 2), (-1, -1, 3, 1, -1), (-4, 15, 1, -1, -4), (2, 2, -6, -2, 2), (12, -11, -13, -1, 2), (-9, 12, 10, 1, -4), (-1, -18, 8, 3, 4), (-1, -5, 15, 7, -11), (7, 1, -3, 2, -8), (3, 14, 4, 4, -17), (-2, 17, -5, -3, -2), (2, 4, -12, -5, 7), (10, -13, -7, 1, 0), (2, -13, -7, -3, 12), (1, -3, 9, 5, -9), (-8, 2, -6, -7, 17), (-4, 11, 13, 5, -14), (-2, -2, 6, 2, -2), (-1, -3, 9, 4, -6), (-4, 0, 0, -2, 6), (7, -3, 9, 8, -18), (4, 2, -6, -1, -1), (7, 5, -15, -4, 2), (3, 3, -9, -3, 3), (3, -1, 3, 3, -7), (11, 1, -3, 4, -14), (3, -1, 3, 3, -7), (3, 3, -9, -3, 3)]
[(2, 0, 0, 1, -3), (9, 3, -9, 0, -6), (4, 2, -6, -1, -1), (-1, 3, -9, -5, 9), (4, 0, 0, 2, -6), (2, 2, -6, -2, 2), (-1, -1, 3, 1, -1), (3, 1, -3, 0, -2), (5, -1, 3, 4, -10), (5, 3, -9, -2, 0), (-3, -3, 9, 3, -3), (6, 0, 0, 3, -9), (5, 1, -3, 1, -5), (-3, 1, -3, -3, 7), (1, -1, 3, 2, -4), (-7, -3, 9, 1, 3), (-4, -2, 6, 1, 1), (3,

In [ ]:
# So far can either get very short vectors in a long time, or not so short vectors quickly
# Thus we have 2 sets of algorithms
# However, we can combine these

# BKZ algorithm (blockwise Korkine-Zolotarev)

# The idea - SVP algorithms scale exponentially with the lattice dimensions
# but at low dimensions, they are reasonably fast
# The LLL algorithm progressively reduces vectors, one by another
# But we can take larger sets of vectors, and find the shortest one

# depending on the tuning of parameters, the quality of the basis, and the running time varies greatly
# in practice, the bases are much better than LLL
# still maintains polynomial running time
# (nevertheless it is slower than LLL)

# comparison

# LLL:
# compute Gram-Schmite vectors
# while improving_the_basis:
#   for every_basis_vector:
#      if can_be_improved_by_another_basis_vector:
#        improve_vector

# BKZ:
# while improving_the_basis:
#   for every_base_partition:
#      compute_SVP
#      improve_partition



In [ ]:
# For CVP, one of the most well known algorithms is the Babai's nearest plane algorithm
# It is actually similar to LLL (also regarding the quality of solution, and time requirements)

In [ ]:
# Input: B - a lattice, t - a vector
# 1. LLL reduce the basis
# 2. b  <- t
#    for base_vector in lattice_basis:
#        c <- <b, base_vector>/<base_vector, base_vector>
#        b <- b - c*base_vector

# What happens in the second step?
